In [1]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "soya_month.csv"
args.stemmer = "lemma"

In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  f = open(args.filename)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
    if not idx % 10:
      print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]

  # X = np.zeros((len(contents), len(dictionary)), dtype=np.int)
  # for idx,i in enumerate(corpus):
  #   for j in i:
  #     X[idx][j[0]] = j[1]



# model = lda.LDA(n_topics=20, n_iter=int(args.num_iter), random_state=1)
# model.fit(X)


# n_top_words = int(args.topwords_count)
# topic_word = model.topic_word_  # odel.components_ also works
# for i, topic_dist in enumerate(topic_word):
#   if args.filename == "sample":
#     topic_words = np.array(dictionary)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#   else:
#     topic_words = [ dictionary[x] for x in np.array(dictionary)[np.argsort(topic_dist)][:-(n_top_words+1):-1] ]
#   print u'Topic {}: {}'.format(i, ' '.join(topic_words))
# doc_topic = model.doc_topic_
# for i in range(10):
#    print u'"{} (top topic: {})"'.format(titles[i], doc_topic[i].argmax())

INFO: Tokenizing articles <0> 
INFO: Tokenizing articles <10> 
INFO: Tokenizing articles <20> 
INFO: Tokenizing articles <30> 
INFO: Tokenizing articles <40> 
INFO: Tokenizing articles <50> 
INFO: Tokenizing articles <60> 
INFO: Tokenizing articles <70> 
INFO: Tokenizing articles <80> 
INFO: Tokenizing articles <90> 
INFO: Tokenizing articles <100> 
INFO: Tokenizing articles <110> 
INFO: Tokenizing articles <120> 
INFO: Tokenizing articles <130> 
INFO: Tokenizing articles <140> 
INFO: Tokenizing articles <150> 
INFO: Tokenizing articles <160> 
INFO: Tokenizing articles <170> 
INFO: Tokenizing articles <180> 
INFO: Tokenizing articles <190> 
INFO: Tokenizing articles <200> 
INFO: Tokenizing articles <210> 
[DEBUG] Length of Texts : 220


In [3]:
pyLDAvis.enable_notebook()
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=10)
pyLDAvis.gensim.prepare(lda, corpus, dictionary)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      23.327290        1       1  0.005132  0.010446
4      16.758962        1       2 -0.018997 -0.002791
6      14.223984        1       3  0.004282 -0.004442
8      11.804191        1       4  0.007663  0.007512
3       9.107943        1       5  0.025502  0.004630
2       8.727948        1       6 -0.000244 -0.012484
0       5.097629        1       7 -0.007766 -0.009745
7       4.741464        1       8  0.018772 -0.019125
5       3.161493        1       9 -0.028909 -0.006746
9       3.049096        1      10 -0.005435  0.032744, topic_info=     Category        Freq      Term  Total  loglift  logprob
623   Default  829.000000         s    829  30.0000  30.0000
5945  Default  552.000000      will    552  29.0000  29.0000
1306  Default  520.000000    device    520  28.0000  28.0000
2684  Default  205.000000    camera    205  27.0000  27.0000
910   Default  243.000000      also    243  26.0000  26.0000
4278  Default  229.000000    galaxy    229  25.0000  25.0000
1937  Default  404.000000       can    404  24.0000  24.0000
121   Default  268.000000        re    268  23.0000  23.0000
4680  Default   80.000000      free     80  22.0000  22.0000
2257  Default  123.000000     month    123  21.0000  21.0000
1057  Default  401.000000         t    401  20.0000  20.0000
4284  Default  254.000000        ll    254  19.0000  19.0000
551   Default  238.000000       one    238  18.0000  18.0000
4099  Default  190.000000    source    190  17.0000  17.0000
3314  Default  155.000000      just    155  16.0000  16.0000
3650  Default  216.000000       get    216  15.0000  15.0000
3025  Default  180.000000   samsung    180  14.0000  14.0000
529   Default   65.000000  internet     65  13.0000  13.0000
5172  Default  186.000000      come    186  12.0000  12.0000
3057  Default  216.000000       new    216  11.0000  11.0000
4478  Default  234.000000      like    234  10.0000  10.0000
380   Default  174.000000      time    174   9.0000   9.0000
916   Default   74.000000      plan     74   8.0000   8.0000
1851  Default   71.000000    update     71   7.0000   7.0000
708   Default   31.000000    celcom     31   6.0000   6.0000
4636  Default   72.000000     redmi     72   5.0000   5.0000
3829  Default  183.000000         u    183   4.0000   4.0000
5337  Default   49.000000      sony     49   3.0000   3.0000
1999  Default  126.000000      year    126   2.0000   2.0000
4613  Default  147.000000     phone    147   1.0000   1.0000
...       ...         ...       ...    ...      ...      ...
1306  Topic10   16.882764    device    520  -0.0475  -4.5207
1937  Topic10   12.123639       can    404  -0.0986  -4.8242
4856  Topic10    3.099551      zoom     48   0.6837  -6.1721
551   Topic10    8.356874       one    238   0.0573  -5.1974
921   Topic10    3.014851      asus     48   0.6575  -6.1983
4680  Topic10    4.513079      free     80   0.3973  -5.9477
2913  Topic10    3.902035   surface     69   0.4554  -6.0375
1761  Topic10    4.023041      hand     87   0.3251  -5.9360
3314  Topic10    5.823684      just    155   0.0695  -5.6141
1057  Topic10    9.059455         t    401  -0.3918  -5.1249
4636  Topic10    3.614831     redmi     72   0.3539  -6.0964
4099  Topic10    6.067746    source    190  -0.0944  -5.5744
3240  Topic10    3.502222      said     73   0.3241  -6.1124
4119  Topic10    3.935467      back     94   0.2034  -5.9803
1999  Topic10    4.501212      year    126   0.0437  -5.8470
3162  Topic10    3.220819       isn     67   0.3451  -6.1772
2490  Topic10    4.350506   battery    122   0.0482  -5.8747
4478  Topic10    5.679839      like    234  -0.3052  -5.5769
3650  Topic10    5.649852       get    216  -0.2788  -5.6305
5945  Topic10    8.169981      will    552  -0.8488  -5.2623
121   Topic10    5.681553        re    268  -0.4809  -5.6169
3585  Topic10    4.316019     first    139  -0.0911  -5.8837
3057  Topic10  